Aprendizaje Automático - Práctica 2
Autores: RICARDO ANTONIO PAZOS VALERO - 100472303 / VICENTE ANTONIO BARBATO - 10043114

# 1. K-Mean
---
Programamos el algoritmo K-mean y comparamos nuestra implementación con la implementación de sklearn